In [1]:
using Flux, DiffEqFlux, Statistics, DifferentialEquations

In [2]:
const bs = 128

128

In [3]:
# function mnist()
#     imgs = Flux.Data.MNIST.images();
#     labels = Flux.Data.MNIST.labels();
#     X = hcat(float.(reshape.(imgs, :))...)
#     Y = Flux.onehotbatch(labels, 0:9)
#     return X, Y
# end

In [4]:
using MLDatasets: MNIST
using MLDataUtils
function loadmnist(batchsize=bs)
    # Use MLDataUtils LabelEnc for natural onehot conversion
    onehot(labels_raw) = convertlabel(LabelEnc.OneOfK, labels_raw, LabelEnc.NativeLabels(collect(0:9)))
    # Load MNIST
    imgs, labels_raw = MNIST.traindata();
    # Process images into (H,W,C,BS) batches
    x_train = reshape(imgs,size(imgs,1),size(imgs,2),1,size(imgs,3))
    x_train = batchview(x_train,batchsize);
    # Onehot and batch the labels
    y_train = onehot(labels_raw)
    y_train = batchview(y_train,batchsize)
    return x_train, y_train
end

loadmnist (generic function with 2 methods)

In [5]:
# X, Y = mnist()

In [6]:
X2, Y2 = loadmnist(128)

┌ Warning: The specified values for size and/or count will result in 96 unused data points
└ @ MLDataPattern C:\Users\Anand\.julia\packages\MLDataPattern\mX21p\src\dataview.jl:204


(SubArray{FixedPointNumbers.Normed{UInt8,8},4,Base.ReshapedArray{FixedPointNumbers.Normed{UInt8,8},4,Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}},Tuple{}},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},UnitRange{Int64}},true}[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 …

In [7]:
# Flux Layer Interface
struct NeuralODE
    model
    tspan
    solver
    args
    kwargs
end

# Optional args and kwargs
NeuralODE(model,tspan,solver,kwargs)= NeuralODE(model,tspan,solver,(),kwargs)
NeuralODE(model,tspan,solver)= NeuralODE(model,tspan,solver,(),())

# Play nice with Flux
Flux.@treelike NeuralODE
Flux.params(n::NeuralODE) = params(n.model)
function (n::NeuralODE)(x)
    return neural_ode(n.model,x,n.tspan,n.solver,n.args...;n.kwargs...)
end

In [8]:
down = Chain(
             Conv((3,3),1=>64,relu,stride=1), GroupNorm(64,64),
             Conv((4,4),64=>64,relu,stride=2,pad=1), GroupNorm(64,64),
             Conv((4,4),64=>64,stride=2,pad=1),
            );
dudt = Chain(
           Conv((3,3),64=>64,relu,stride=1,pad=1),
           Conv((3,3),64=>64,relu,stride=1,pad=1)
          ) ;
fc = Chain(GroupNorm(64,64),
           x->relu.(x),
           MeanPool((6,6)),
           x -> reshape(x,(64,bs)),
           Dense(64,10)
          );

In [9]:
solver_kwargs = Dict(:save_start=>false,:save_everystep=>false, :reltol=>1e-3, :abstol=>1e-3)
neural_ode_layer = NeuralODE(dudt, (0.f0,1.f0), Tsit5(), solver_kwargs)

NeuralODE(Chain(Conv((3, 3), 64=>64, NNlib.relu), Conv((3, 3), 64=>64, NNlib.relu)), (0.0f0, 1.0f0), Tsit5(), (), Dict{Symbol,Real}(:save_everystep=>false,:reltol=>0.001,:abstol=>0.001,:save_start=>false))

In [10]:
model = Chain(
              down,             #(28,28,1,BS) -> (6,6,64,BS)
              neural_ode_layer, #(6,6,64,BS) -> (6,6,64,BS)
              fc                #(6,6,64,BS) -> (10, BS)
             )

Chain(Chain(Conv((3, 3), 1=>64, NNlib.relu), GroupNorm(64), Conv((4, 4), 64=>64, NNlib.relu), GroupNorm(64), Conv((4, 4), 64=>64)), NeuralODE(Chain(Conv((3, 3), 64=>64, NNlib.relu), Conv((3, 3), 64=>64, NNlib.relu)), (0.0f0, 1.0f0), Tsit5(), (), Dict{Symbol,Real}(:save_everystep=>false,:reltol=>0.001,:abstol=>0.001,:save_start=>false)), Chain(GroupNorm(64), getfield(Main, Symbol("##4#6"))(), MeanPool((6, 6), pad = (0, 0, 0, 0), stride = (6, 6)), getfield(Main, Symbol("##5#7"))(), Dense(64, 10)))

In [11]:
# relu
x_m = model(X2[1])

Tracked 10×128 Array{Float32,2}:
 -0.48809    -0.455704   -0.422409  …  -0.488604  -0.465925  -0.514481 
  0.654322    0.657156    0.696695      0.663026   0.584065   0.613291 
 -0.412922   -0.418761   -0.443609     -0.456238  -0.41203   -0.384866 
  0.227947    0.225564    0.269854      0.202474   0.213786   0.182721 
 -0.34781    -0.328298   -0.345226     -0.32616   -0.305509  -0.30709  
 -0.281498   -0.330351   -0.257438  …  -0.311515  -0.286513  -0.351317 
  0.575244    0.524117    0.540353      0.530706   0.577402   0.445278 
  1.42405     1.39749     1.43456       1.41738    1.4036     1.42155  
  0.0434969   0.0998035   0.126664      0.122363   0.107688   0.0719847
  0.815427    0.886549    0.809267      0.852142   0.864452   0.893978 

In [12]:
loss(x, y) = Flux.logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [13]:
loss(X2[1], Y2[1])

2.4444807f0 (tracked)

In [14]:
classify(x) = argmax.(eachcol(x))

classify (generic function with 1 method)

In [15]:
function accuracy(model,data; n_batches=100)
    total_correct = 0
    total = 0
    for (x,y) in collect(data)[1:n_batches]
        target_class = classify(cpu(y))
        predicted_class = classify(cpu(model(x)))
        total_correct += sum(target_class .== predicted_class)
        total += length(target_class)
    end
    return total_correct/total
end

accuracy (generic function with 1 method)

In [ ]:
accuracy(model, zip(X2,Y2))

In [ ]:
opt=ADAM()
Flux.train!(loss,params(model),ncycle(zip(X2,Y2),100),opt, cb=cb)